__ Assignment 2__ 

2) Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month.

In this exercise you will use the temperature-readings.csv file. The output should contain the following information:
__Year, month, count__

For each question include the following data in the report and sort it as shown:
- year, month, value ORDER BY value DESC 
- year, month, value ORDER BY value DESC

In [1]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

In [2]:
path_unifolder_temp_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv"


In [5]:
# read and create rdd, this needs to be tranfsormed to data frame later on
temps= sc.textFile(path_unifolder_temp_tiny)
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], time=p[2],  Temp=float(p[3]), quality=p[4], month=p[1].split("-")[1]))
tempReadings.take(10)


[Row(Temp=6.8, date='2013-11-01', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.8, date='2013-11-01', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=5.8, date='2013-11-02', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=-1.1, date='2013-11-02', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=-0.2, date='2013-11-03', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.6, date='2013-11-03', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=6.5, date='2013-11-04', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.1, date='2013-11-04', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=4.2, date='2013-11-05', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.2, date='2013-

In [4]:
# create a data frame from rdd
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show(10)

+----+----------+-----+-------+-------+--------+----+
|Temp|      date|month|quality|station|    time|year|
+----+----------+-----+-------+-------+--------+----+
| 6.8|2013-11-01|   11|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01|   11|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02|   11|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02|   11|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03|   11|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03|   11|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04|   11|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04|   11|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05|   11|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05|   11|      G| 102170|18:00:00|2013|
+----+----------+-----+-------+-------+--------+----+
only showing top 10 rows



__SQL query__

In [31]:
# SQL can be run over DataFrames that have been registered as a table.
# register DataFrame as a table
schemaTempReadings.registerTempTable("tempReadingsTable")

In [49]:
# we want to know the max and min temp for a specifitc time interval + min temp of 10
# time interval >= 1950 & <= 2014
# temp > 10
result = sqlContext.sql(
    "SELECT year, month, count(Temp) as count  \
    FROM tempReadingsTable\
    WHERE year >= 1950 and year <= 2014 and Temp > 10\
    GROUP BY year, month\
    ORDER BY count DESC")
result.show(10)

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|1959|   07|  111|
|1958|   07|  110|
|1960|   07|  110|
|1957|   07|  109|
|1960|   06|  108|
|1956|   07|  107|
|1959|   08|  101|
|1960|   08|   98|
|1958|   08|   96|
|1957|   08|   96|
+----+-----+-----+
only showing top 10 rows



*Distinct version*

In [48]:
result_distinct= sqlContext.sql(
    "SELECT year, month, count(DISTINCT Temp) as count  \
    FROM tempReadingsTable\
    WHERE year >= 1950 and year <= 2014 and Temp > 10\
    GROUP BY year, month\
    ORDER BY count DESC")
result_distinct.show(10)

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|1960|   06|   79|
|1959|   08|   76|
|1956|   07|   75|
|1959|   07|   74|
|1959|   06|   70|
|1957|   07|   69|
|1958|   07|   64|
|1960|   07|   61|
|1958|   06|   60|
|1960|   08|   59|
+----+-----+-----+
only showing top 10 rows

